In [33]:
!pip install google

Defaulting to user installation because normal site-packages is not writeable
     -------------------------------------- 45.3/45.3 kB 555.8 kB/s eta 0:00:00


In [34]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [8]:
!pip install torch

Defaulting to user installation because normal site-packages is not writeable
     -------------------------------------- 204.1/204.1 MB 5.0 MB/s eta 0:00:00
     ---------------------------------------- 6.2/6.2 MB 6.3 MB/s eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
with open('C:\\Users\\NiKordzakhia\\Desktop\\Transformer-BasedGPTModel\\vefxistyaosani.txt', 'r', encoding='utf-8') as f:
  data = f.read()

In [4]:
print(f'Length of dataset in characters: {len(data)}')

Length of dataset in characters: 329322


In [5]:
chars = sorted(list(set(data)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !",-.:;?«»აბგდევზთიკლმნოპჟრსტუფქღყშჩცძწჭხჯჰ–—“”
49


In [6]:
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[c] for c in l])

In [9]:
import torch
encoded_data = encode(data)
data = torch.tensor(encoded_data, dtype = torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([329322]) torch.int64
tensor([17, 16, 32, 42, 20, 29, 30, 35, 12, 25, 29, 12, 24, 20,  0, 15, 12, 29,
        12, 40, 35, 20, 29, 20,  0,  0, 28, 25, 23, 16, 22, 23, 12, 24,  1, 36,
        16, 33, 23, 24, 12,  1, 29, 12, 23, 35, 12, 28, 25,  1, 39, 12, 22, 20,
        19, 12,  1, 23, 20, 19,  1, 39, 22, 20, 16, 28, 20, 19, 12,  4,  0, 18,
        16, 14, 12, 28, 15, 23, 25,  1, 12, 28, 29, 24, 20,  1, 29, 31, 22, 20,
        19, 12,  1, 35, 17, 24, 12,  1, 18, 16])


In [10]:
n = int(len(data) * 0.9)
train_data = data[:n]
valid_data = data[n:]

In [11]:
block_size = 8
train_data[1:block_size + 1]

tensor([16, 32, 42, 20, 29, 30, 35, 12])

In [12]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for i in range(block_size):
  context = x[:i+1]
  target = y[i]
  print(f"Context: {context}; Target: {target}.")

Context: tensor([17]); Target: 16.
Context: tensor([17, 16]); Target: 32.
Context: tensor([17, 16, 32]); Target: 42.
Context: tensor([17, 16, 32, 42]); Target: 20.
Context: tensor([17, 16, 32, 42, 20]); Target: 29.
Context: tensor([17, 16, 32, 42, 20, 29]); Target: 30.
Context: tensor([17, 16, 32, 42, 20, 29, 30]); Target: 35.
Context: tensor([17, 16, 32, 42, 20, 29, 30, 35]); Target: 12.


In [13]:
torch.manual_seed(1)
batch_size = 4
block_size = 8

def get_batch(split):
  data = train_data if split == 'train' else valid_data
  ix = torch.randint(len(data) - block_size, (batch_size, )) # torch.randint(low, high, size)
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x, y

xb, yb = get_batch('train')

In [14]:
xb, yb

(tensor([[12,  1, 12, 42, 15, 12,  1, 36],
         [40, 35, 20, 24, 16,  4,  0, 28],
         [20, 24, 12, 42, 16,  4,  1,  5],
         [ 1, 31, 18, 25, 23, 25,  4,  1]]),
 tensor([[ 1, 12, 42, 15, 12,  1, 36, 12],
         [35, 20, 24, 16,  4,  0, 28, 12],
         [24, 12, 42, 16,  4,  1,  5,  1],
         [31, 18, 25, 23, 25,  4,  1, 29]]))

In [15]:
for i in range(batch_size):
  for j in range(block_size):
    context = xb[i, :j+1]
    target = yb[i,j]
    print(f'When the input is {context.tolist()}; target is {target}')

When the input is [12]; target is 1
When the input is [12, 1]; target is 12
When the input is [12, 1, 12]; target is 42
When the input is [12, 1, 12, 42]; target is 15
When the input is [12, 1, 12, 42, 15]; target is 12
When the input is [12, 1, 12, 42, 15, 12]; target is 1
When the input is [12, 1, 12, 42, 15, 12, 1]; target is 36
When the input is [12, 1, 12, 42, 15, 12, 1, 36]; target is 12
When the input is [40]; target is 35
When the input is [40, 35]; target is 20
When the input is [40, 35, 20]; target is 24
When the input is [40, 35, 20, 24]; target is 16
When the input is [40, 35, 20, 24, 16]; target is 4
When the input is [40, 35, 20, 24, 16, 4]; target is 0
When the input is [40, 35, 20, 24, 16, 4, 0]; target is 28
When the input is [40, 35, 20, 24, 16, 4, 0, 28]; target is 12
When the input is [20]; target is 24
When the input is [20, 24]; target is 12
When the input is [20, 24, 12]; target is 42
When the input is [20, 24, 12, 42]; target is 16
When the input is [20, 24, 12,

In [16]:
xb[:2,:]

tensor([[12,  1, 12, 42, 15, 12,  1, 36],
        [40, 35, 20, 24, 16,  4,  0, 28]])

In [17]:
token_embedding_table = nn.Embedding(vocab_size, 2)
token_embedding_table(xb[:2,:])

NameError: name 'nn' is not defined

In [18]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1)

class BiggramLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, idx, targets = None):
    logits = self.token_embedding_table(idx) # (B, T, C) ==> Batch, Time, Channels; C ==> Number of classes (vocabulary size for classification).
    if targets == None:
      loss = None
    else:
      B, T, C = logits.shape
      #print("Logits shape: ", logits.shape)
      #print("Targets shape: ", targets.shape)
      #print('-----------------------------')
      logits = logits.view(B*T, C)
      #print("Logits shape after: ", logits.shape)
      targets = targets.view(B*T)
      #print("Targets shape after: ", targets.shape)
      loss = F.cross_entropy(logits, targets) # Expects (B, C, T) ==> CrossEntropy Loss computes loss per token, then averages over all 32 tokens.
    return logits, loss
  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
      logits, loss = self(idx)
      print("Logits before: ", logits.shape)
      logits = logits[:, -1,:] # Becomes (B, C) we need last token logits because when sentence comes we do not care about history but what was last word instead.
      #The model generates text one token at a time, and each token generated is influenced by the previous tokens. But for each step, the context for generating the next token is only the most recent token—not the entire sequence.
      print("Logits after: ", logits.shape)
      probs = F.softmax(logits, -1)
      idx_next = torch.multinomial(probs, num_samples=1) # (B, 1) ; If probs = [0.1, 0.4, 0.3, 0.2], then calling torch.multinomial(probs, num_samples=1) will randomly select one index from this list (e.g., index 1, index 2, etc.), with the likelihood of each index being selected proportional to its probability in the list.
      idx = torch.cat((idx, idx_next), dim = 1)
    return idx

In [19]:
m = BiggramLanguageModel(vocab_size)
logits, loss = m.forward(xb, yb)
print(logits.shape)
print(loss)

torch.Size([32, 49])
tensor(4.2583, grad_fn=<NllLossBackward0>)


In [20]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens = 2)[0].tolist()))

Logits before:  torch.Size([1, 1, 49])
Logits after:  torch.Size([1, 49])
Logits before:  torch.Size([1, 2, 49])
Logits after:  torch.Size([1, 49])

;



In [21]:
optimizer = torch.optim.AdamW(m.parameters(), lr = 1e-3)

In [26]:
batch_size = 32
for step in range(1000):
  xb, yb = get_batch('train')
  logits, loss = m(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()
  print(loss.item())

2.710899829864502
2.5852248668670654
2.6015992164611816
2.6368637084960938
2.6903347969055176
2.7045798301696777
2.680438756942749
2.5757222175598145
2.657632827758789
2.693932294845581
2.628763437271118
2.5537397861480713
2.6995577812194824
2.673140525817871
2.6172282695770264
2.502389669418335
2.638719320297241
2.6107304096221924
2.583033800125122
2.616863965988159
2.5899806022644043
2.645350217819214
2.641684055328369
2.5644443035125732
2.570164918899536
2.635409116744995
2.5563323497772217
2.546370506286621
2.5671591758728027
2.5782039165496826
2.6661882400512695
2.651338815689087
2.6852824687957764
2.6388356685638428
2.6901488304138184
2.598951578140259
2.6131062507629395
2.692652702331543
2.6567282676696777
2.6476874351501465
2.6257824897766113
2.7189481258392334
2.6811041831970215
2.5137956142425537
2.5868520736694336
2.5966243743896484
2.6583597660064697
2.616643190383911
2.6178157329559326
2.550940990447998
2.6226892471313477
2.6869449615478516
2.5333750247955322
2.63262677192

In [27]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens = 100)[0].tolist()))

Logits before:  torch.Size([1, 1, 49])
Logits after:  torch.Size([1, 49])
Logits before:  torch.Size([1, 2, 49])
Logits after:  torch.Size([1, 49])
Logits before:  torch.Size([1, 3, 49])
Logits after:  torch.Size([1, 49])
Logits before:  torch.Size([1, 4, 49])
Logits after:  torch.Size([1, 49])
Logits before:  torch.Size([1, 5, 49])
Logits after:  torch.Size([1, 49])
Logits before:  torch.Size([1, 6, 49])
Logits after:  torch.Size([1, 49])
Logits before:  torch.Size([1, 7, 49])
Logits after:  torch.Size([1, 49])
Logits before:  torch.Size([1, 8, 49])
Logits after:  torch.Size([1, 49])
Logits before:  torch.Size([1, 9, 49])
Logits after:  torch.Size([1, 49])
Logits before:  torch.Size([1, 10, 49])
Logits after:  torch.Size([1, 49])
Logits before:  torch.Size([1, 11, 49])
Logits after:  torch.Size([1, 49])
Logits before:  torch.Size([1, 12, 49])
Logits after:  torch.Size([1, 49])
Logits before:  torch.Size([1, 13, 49])
Logits after:  torch.Size([1, 49])
Logits before:  torch.Size([1, 14,

In [30]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cu118
     - 109.0 kB ? 0:00:00


ERROR: THESE PACKAGES DO NOT MATCH THE HASHES FROM THE REQUIREMENTS FILE. If you have updated the package versions, please update the hashes. Otherwise, examine the package contents carefully; someone may have tampered with them.
    torchvision from https://download.pytorch.org/whl/cu118/torchvision-0.21.0%2Bcu118-cp39-cp39-win_amd64.whl#sha256=521ffecc39798b86e0b51960527f56d54c9ccd8e2b5435768283da5f7de5da56:
        Expected sha256 521ffecc39798b86e0b51960527f56d54c9ccd8e2b5435768283da5f7de5da56
             Got        52a2e90401af430829b99bd6639d037681ec4a50a2452ebec9468b14553bab16



In [31]:
if torch.cuda.is_available():
  print('available')